#Продвинутый Python, ДЗ-2

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно использовать pandas

In [278]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [ ]:
#!pip install wget
#перед сдачей это закомментить

In [279]:
# Код, который будет в каждом тесте, названия не менять
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (10 баллов)

Определите:

* Число товаров

* Среднюю стоимость товара (стоимость товара = среднее от цен в датасете items)

в разрезе категорий (все категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
category | products | price

value    | value      | value
```

**Обратите внимание:**

Для категории portateis_cozinha_e_preparadores_de_alimentos перевод portable kitchen and food preparers

Для категории pc_gamer перевод PC Gamer

Для них нужно отдельно добавить перевод

In [153]:
%%write_and_run task_1.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_1(translation, items, products):

    prices = items[['product_id', 'price']].groupby(['product_id']).agg({'price' : 'mean'})

    new_table = pd.merge(products[['product_id', 'product_category_name']], prices, on='product_id')

    agg_func = {'product_id': pd.Series.nunique, 'price' : 'mean'}
    items_info = new_table.groupby(['product_category_name']).agg(agg_func).rename(columns={'product_id': 'products'})

    add_tr = pd.DataFrame([['portateis_cozinha_e_preparadores_de_alimentos', 'portable kitchen and food preparers'], ['pc_gamer', 'PC Gamer']], columns=list(translation.columns))
    translation = pd.concat([translation, add_tr])

    table_translated = pd.merge(items_info, translation, on='product_category_name')[['product_category_name_english', 'products', 'price']].rename(columns={'product_category_name_english': 'category'})

    return table_translated
    

## Задание 2 (10 баллов)

Определите для каждого продавца основную категорию их продаж (категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
seller_id | category

value    | value
```




In [311]:
%%write_and_run task_2.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_2(translation, products, items):
    # add_tr = pd.DataFrame([['portateis_cozinha_e_preparadores_de_alimentos', 'portable kitchen and food preparers'], ['pc_gamer', 'PC Gamer']], columns=list(translation.columns))
    # translation = pd.concat([translation, add_tr])
    new_table = pd.merge(products[['product_id', 'product_category_name']], items[['product_id', 'seller_id']].dropna(), on='product_id', how='right')

    agg_func = {'product_id': 'count'}
    items_info = new_table.groupby(['seller_id', 'product_category_name']).agg(agg_func).rename(columns={'product_id': 'products'}).reset_index()
    
    items_info = items_info.sort_values('products', ascending=False).drop_duplicates(subset='seller_id')[['seller_id', 'product_category_name']].reset_index()
    table_translated = pd.merge(items_info, translation, on='product_category_name', how='left')[['seller_id', 'product_category_name_english']].rename(columns={'product_category_name_english': 'category'})
    return table_translated

[3] [4] [4] [4] [8] [8] [10] [12] [13] [14] [15] [16] [17] [17] [18] [18] [19] [20] [21] [22] [23] [28] [36] [43] [44] [53] [54] [59] [64] [69] [77] [86] [97] [102] [118] [125] [155] [212] [222] [256] [289] [310] 

## Задание 3 (10 баллов)

Выведите долю потраченных денег по каждому штату (потраченные деньги - сумма денег по доставленным заказам, сумма по price и freight_value)

*Примечание:* разбивка по штатам - по штату покупателя, процент - число от 0 до 1


Табличка, которая у вас должна получиться:

```
state | perc

value | value
```

In [351]:
%%write_and_run task_3.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_3(orders, customers, items):
    # orders -> order_id, customer_id, order_status
    # items -> order_id, price, freight_value
    # customers -> customer_id, customer_state

    # items_prices -> order_id, customer_id, order_status, price, freight_value
    items_prices = pd.merge(orders, items[['order_id', 'price', 'freight_value', 'order_item_id']], on='order_id')
    items_prices.loc[items_prices.order_item_id != 1, 'freight_value'] = 0

    # items_prices -> order_id, customer_id, price, freight_value
    items_prices = items_prices[(items_prices['order_status'] == 'delivered')][['order_id', 'customer_id', 'price', 'freight_value']]
    items_prices['sum'] = items['price'] + items['freight_value']

    # items_prices -> order_id, customer_id,  sum
    items_prices = items_prices[['order_id', 'customer_id', 'sum']]
    sum_ = items_prices.sum()
    
    # new_table -> customer_state, sum
    new_table = pd.merge(customers[['customer_id', 'customer_state']], items_prices, on='customer_id')[['customer_state', 'sum']]
    new_table = new_table.groupby('customer_state').agg({'sum': lambda x: sum(x) / sum_[-1]}).reset_index().rename(columns={'customer_state' : 'state', 'sum' : 'perc'})

    return new_table


## Задание 4 (10 баллов)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

А также определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [338]:
%%write_and_run task_4.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_4(items, orders, customers):
    # items -> order_id, price, freight_value
    # orders -> order_id, order_item_id
    # customers -> customer_unique_id

    orders_grouped = items[['order_id', 'price', 'freight_value']].groupby('order_id').agg({'price': 'sum', 'freight_value': 'sum'}).reset_index()
    price_avg = orders_grouped[['price']].mean().price
    freight_avg = orders_grouped[['freight_value']].mean().freight_value

    orders_counts = items[['order_id', 'order_item_id']].groupby('order_id').agg({'order_item_id': 'count'}).reset_index()
    items_avg = orders_counts[['order_item_id']].mean().order_item_id

    customers_orders = customers[['customer_unique_id', 'customer_id']].groupby('customer_unique_id').agg({'customer_id': 'count'}).reset_index()
    orders_avg = customers_orders[['customer_id']].mean().customer_id

    return (price_avg, freight_avg, items_avg, orders_avg)
    

## Задание 5 (10 баллов)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням в период с апреля 2017 по апрель 2018 года

Все манипуляции с датой необходимо сделать с помощью datetime и dateutil


Табличка, которая у вас должна получиться:

```
date | csat

"YYYY-MM-DD" | value
```

In [340]:
%%write_and_run task_5.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_5(reviews):
    reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
    reviews['date'] = reviews.review_creation_date.apply(lambda x: x.strftime('%Y-%m-%d'))
    reviews = reviews[(reviews.date >= '2017-04-01') & (reviews.date <= '2018-04-30')]
    res = reviews.groupby('date').review_score.mean().reset_index().rename(columns={'review_score':'csat'})
    return res

## Задание 6 (10 баллов)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по числу дней ответа) и какая средняя оценка

Все манипуляции со временем нужно делать через datetime и dateutil

Табличка, которая у вас должна получиться:

```
days | csat | orders

value | value | value
```

Результаты должны быть отсориртированы по дня по возрастанию

In [317]:
%%write_and_run task_6.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_6(reviews):
    reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
    reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])
    reviews['days'] = reviews['review_answer_timestamp'] - reviews['review_creation_date']
    reviews['days'] = reviews.days.apply(lambda x: x.days)
    agg_func = {'order_id':'count','review_score':'mean'}
    res = reviews.groupby('days').agg(agg_func).reset_index().rename(columns={'review_score':'csat', 'order_id':'orders'}).sort_values('days')
    return res


## Задание 7 (10 баллов)

Выделите все заказы, где не проставлено поле order_delivered_customer_date. Замените его на дату '2999-12-31'

In [292]:
%%write_and_run task_7.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_7(orders):
    orders['order_delivered_customer_date'] = orders['order_delivered_customer_date'].fillna('2999-12-31')
    return orders

## Задание 8 (30 баллов)

Определите топ-10 продавцов, у которых больше 100 заказов, отсылающие чаще всего свою посылку в другие регионы (считаются только доставленные заказы)

Чаще всего отсылают = самый большой процент отправленных заказов в другой штат

Табличка, которая у вас должна получиться:

```
seller_id | share

value | value
```

In [315]:
%%write_and_run task_8.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    orders = orders[orders['order_status'] == 'delivered']
    
    df = pd.merge(orders[['order_id', 'customer_id']], items[['order_id', 'seller_id']], on='order_id', how='inner')
    df = pd.merge(df, customers[['customer_id', 'customer_state']], on='customer_id', how='inner').drop_duplicates(subset=['seller_id', 'order_id'])

    df = pd.merge(df[['customer_state', 'seller_id']], sellers[['seller_id', 'seller_state']], on='seller_id')
    df['share'] = df.apply(lambda x: x.customer_state != x.seller_state, axis=1)
    agg_func={'share':'sum', 'customer_state':'count'}
    res = df.groupby('seller_id').agg(agg_func).reset_index()
    res['share'] = res.apply(lambda x: x.share/x.customer_state, axis=1)
    res = res[res.customer_state > 100][['seller_id','share']].sort_values('share', ascending=False)[:10]
    return res
